In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras.utils import to_categorical

## Step 2 - Prepare dataset & create baseline model.

In [3]:
house_dataset = pd.read_csv('./housing.csv', header=None, delim_whitespace=True)
house_dataset

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [4]:
house_dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [5]:
house_dataset.shape

(506, 14)

In [6]:
X = house_dataset.iloc[:,0:13]
Y = house_dataset.iloc[:,13]

In [7]:
def baseline_model():
    model = Sequential()
    model.add(Dense(13,activation='relu',input_shape=(13,)))
    model.add(Dense(13,activation='relu'))
    model.add(Dense(1))
    # Compile.
    model.compile(loss='MSE',optimizer='adam',metrics=['accuracy'])
    return model

In [8]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [9]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Results: -38.73 (28.50) MSE


## Step 3: Modeling The Standardized Dataset

In [10]:
# evaluate model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Standardized: -21.74 (27.57) MSE


### Step 3.2 - use a Sigmoid or similar activation function on the output layer to narrow output predictions to the same range

In [11]:
def new_baseline_model():
    model = Sequential()
    model.add(Dense(13,activation='relu',input_shape=(13,)))
    model.add(Dense(13,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    # Compile.
    model.compile(loss='MSE',optimizer='adam',metrics=['accuracy'])
    return model

In [12]:
np.random.seed(seed)
estimators = []
estimators.append(('mlp', KerasRegressor(build_fn=new_baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Standardized: -549.12 (273.00) MSE


## Step 4 - Tune The Neural Network Topology.

### Step 4.1 - Evaluate a Deeper Network Topology.

In [13]:
def larger_model():
    model = Sequential()
    model.add(Dense(13,activation='relu',input_shape=(13,)))
    model.add(Dense(13,activation='relu'))
    model.add(Dense(6,activation='relu'))
    model.add(Dense(1))
    # Compile.
    model.compile(loss='MSE',optimizer='adam',metrics=['accuracy'])
    return model

In [14]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -21.52 (25.84) MSE


### Step 4.2. Evaluate a Wider Network Topology

In [15]:
def wider_model():
    model = Sequential()
    model.add(Dense(13,activation='relu',input_shape=(13,)))
    model.add(Dense(20,activation='relu'))
    model.add(Dense(1))
    # Compile.
    model.compile(loss='MSE',optimizer='adam',metrics=['accuracy'])
    return model

In [16]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Wider: -21.42 (21.47) MSE


## Step 5. Really Scaling up: developing a model that overfits

In [17]:
def overfit_model():
    model = Sequential()
    model.add(Dense(13,activation='relu',input_shape=(13,)))
    model.add(Dense(30,activation='relu'))
    model.add(Dense(30,activation='relu'))
    model.add(Dense(20,activation='relu'))
    model.add(Dense(1))
    # Compile.
    model.compile(loss='MSE',optimizer='adam',metrics=['accuracy'])
    return model

In [18]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=overfit_model, epochs=150, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Wider: -35.57 (31.48) MSE


## Step 6. Tuning the Model

In [19]:
def baseline_model():
    model = Sequential()
    model.add(Dense(13,activation='relu',input_shape=(13,)))
    model.add(Dense(13,activation='relu'))
    model.add(Dense(1))
    # Compile.
    model.compile(loss='MSE',optimizer='adam',metrics=['accuracy'])
    return model

# Result: Standardized: -20.45 (25.99) MSE

## Step 7. Rewriting the code using the Keras Functional API

In [20]:
def functional_api():
    model = Sequential()
    model.add(Dense(13,activation='relu',input_shape=(13,)))
    model.add(Dense(13,activation='relu'))
    model.add(Dense(1))
    # Compile.
    model.compile(loss='MSE',optimizer='adam',metrics=['accuracy'])
    return model

In [21]:
model = functional_api()
history = model.fit(X,Y,epochs=50, batch_size=5, verbose=False)
history_dict = history.history
acc_values = history_dict['loss']
print("Result: %.2f%% (%.2f%%)" % (np.mean(acc_values)*100, np.std(acc_values)*100))

Result: 10082.18% (36258.73%)


## Step 8. Rewriting the code by doing Model Subclassing

In [22]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.dense1 = Dense(13,activation='relu')
        self.dense2 = Dense(13,activation='relu')
        self.dense3 = Dense(1)
    
    def call(self,inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)

def subclass_model():
    inputs = tf.keras.Input(shape=(13,))
    mymodel = MyModel()
    outputs = mymodel.call(inputs)
    # Keras Model.
    model = tf.keras.Model(inputs,outputs)
    # Compile.
    model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])
    return model

In [23]:
mymodel = subclass_model()
history = mymodel.fit(X,Y,epochs=50, batch_size=5, verbose=True)
history_dict = history.history
acc_values = history_dict['loss']
print("Result: %.2f%% (%.2f%%)" % (np.mean(acc_values)*100, np.std(acc_values)*100))

Train on 506 samples
Epoch 1/50
506/506 [==============================] - 1s 3ms/sample - loss: 178.8052 - acc: 0.0000e+00
Epoch 2/50
506/506 [==============================] - 0s 701us/sample - loss: 70.5227 - acc: 0.0000e+00
Epoch 3/50
506/506 [==============================] - 0s 704us/sample - loss: 64.9343 - acc: 0.0000e+00
Epoch 4/50
506/506 [==============================] - 0s 723us/sample - loss: 64.4175 - acc: 0.0000e+00
Epoch 5/50
506/506 [==============================] - 0s 718us/sample - loss: 54.6050 - acc: 0.0000e+00
Epoch 6/50
506/506 [==============================] - 0s 721us/sample - loss: 55.0880 - acc: 0.0000e+00
Epoch 7/50
506/506 [==============================] - 0s 694us/sample - loss: 48.2680 - acc: 0.0000e+00
Epoch 8/50
506/506 [==============================] - 0s 711us/sample - loss: 48.5837 - acc: 0.0000e+00
Epoch 9/50
506/506 [==============================] - 0s 707us/sample - loss: 49.3413 - acc: 0.0000e+00
Epoch 10/50
506/506 [=======================

## Step 9. Rewriting the code without using scikit-learn

In [24]:
k=10
num_val_samples = len(X) // k
num_epochs = 50
all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = Y[i * num_val_samples: (i + 1) * num_val_samples]
#     print(val_data,val_targets)
#     partial_train_data =np.concatenate([X[:i * num_val_samples],X[(i + 1) * num_val_samples:]],axis=0)
#     partial_train_targets =np.concatenate([Y[:i * num_val_samples],Y[(i + 1) * num_val_samples:]],axis=0)
    model = baseline_model()
    model.fit(val_data, val_targets,epochs=num_epochs, batch_size=5, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [25]:
print(f"K-Fold Results: {str(np.mean(all_scores)*100)}%,{str(np.std(all_scores)*100)}%.")
all_scores

K-Fold Results: 0.0%,0.0%.


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]